In [ ]:
import holoviews as hv
import hvplot.xarray  # noqa: F401
import numpy as np
import xarray as xr
from fsspec.implementations.http import HTTPFileSystem
from holoviews import opts
import json
import rioxarray

from emit_tools import emit_xarray
from utils import get_rgb_dataset, hv_stream_to_rio_geometries, get_earthdata_token

In [ ]:
# See README.md for instructions on how to get an Earthdata token
token = get_earthdata_token()

In [ ]:
%%time
# Loading data can take around 3-4 minutes on a 100 Mbps connection

# Refer to the README.md for instructions on how to find granule IDs
granule = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + granule + "/" + granule + ".nc"
http_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(http_url))
ds

In [ ]:
%%capture --no-stdout
# Create an RGB dataset for rendering

from utils import get_rgb_dataset


# Select wavelengths to be displayed
r = 2100
g = 950
b = 490

# Select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.4

## End configuration area ##

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], brightness)

In [ ]:
# Clean up empty bands.
ds = ds.fillna(np.nan).where(ds.reflectance!=-0.01)

In [ ]:
# Mask out land
high = ds.reflectance.sel(bands=450, method="nearest")
low = ds.reflectance.sel(bands=1275, method="nearest")

mask = ((high - low) / (high + low)) > 0.0
ds = ds.fillna(np.nan).where(mask)

In [ ]:
# Limit the number of drawn polygons
POLY_LIMIT = 5

color_cycle = hv.Cycle('Category10')
colors = [color_cycle[i] for i in range(5)]

# RGB image/map
map = ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

# Set up a holoviews points array to enable plotting of the clicked points
xmid = ds.longitude.values[int(len(ds.longitude) / 2)]
ymid = ds.latitude.values[int(len(ds.latitude) / 2)]
polygons = hv.Polygons(
    [],
    kdims=["xs", "ys"],
)

polygons_stream = hv.streams.PolyDraw(
    # data=polygons.columns(),
    source=polygons,
    num_objects=POLY_LIMIT,
    styles={'fill_color': color_cycle.values[0:POLY_LIMIT]}
)

# Plot the Map and Dynamic Map side by side
(map * polygons)

In [ ]:
# Build a spectral plot for each of the drawn polygons.
# This takes a while.

plots = []

geometries = hv_stream_to_rio_geometries(polygons_stream.data)
export = []

for i, geometry in enumerate(geometries):
    data = ds.reflectance.rio.clip(geometry, drop=False)
    hv_data = hv.Dataset(data, kdims=["bands", "latitude", "longitude"], vdims=["reflectance"])
    agg = hv_data.aggregate("bands", np.nanmean, spreadfn=np.nanstd)
    
    data = [i, json.dumps(geometry)] + list(agg.data.reflectance.values) + list(agg.data.reflectance_nanstd.values)
    export.append(data)                                                    

    plots.append(
        (hv.Spread(agg) * hv.Curve(agg, label=f"{i}"))
    )

hv.Overlay(plots).opts(
    opts.Spread(color=color_cycle),
    opts.Curve(color=color_cycle),
    opts.Overlay(show_title=False, frame_width=600, show_legend=True)
)

In [ ]:
import csv 

data = polygons_stream.data
wavelengths = ds.wavelengths.values

rows = [["id", "geom"] + [f"{i}_m" for i in wavelengths] + [f"{i}_std" for i in wavelengths]]

rows = rows + export
    
len(rows)

In [ ]:
with open('polygon_data.csv', 'w') as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(rows)